<a href="https://colab.research.google.com/github/Tecnologico-de-Monterrey-MNA/nlp-2023_Equipo-6/blob/main/Equipo_06_Semana_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objetivo de la Actividad**


> Trabajar con estos modelos pre-entrenados, generando el vocabulario a partir de tu conjunto de datos de entrenamiento.

Para cada palabra de tu vocabulario, podrás sustituirlo por su correspondiente
vector continuo. En caso de que no exista el vector para una palabra en particular, se puede eliminar dicha palabra, o bien sustituirla por el vector continuo más cercano.

En esta actividad deberás aplicar esta segunda opción.

-----

*   Existen diversas propuestas para utilizar dichos vectores continuos como entrada para modelos de aprendizaje automático. En particular, en esta actividad cada enunciado será sustituido por el vector promedio de todos los tokens que lo forman.

**Modelos:**

Modelo de vectores **continuos/embebidos FastText**, es decir, el modelo desarrollado por Facebook en 2016.

## **Pregunta 1**

Descarga los **3 archivos de Canvas**. En particular, el archivo de datos de **IMDb** ya no requiere transformarse **para obtener sus 1000 registros**. Al cargar los datos de los tres archivos deberás tener un **DataFrame de Pandas de 3000 registros**, con sus etiquetas. Los archivos los encuentras en Canvas y se llaman: **amazon5.txt, imdb5.txt, yelp5.txt.**

In [1]:
import pandas as pd
import numpy as np

import re
import string

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer, WordNetLemmatizer, RegexpStemmer
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Jorge
[nltk_data]     Pedroza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
# !gunzip cc.en.300.vec.gz

#**Aplicando NLTK**

In [3]:
nltk.download('punkt')        # Tokenizador que ayuda a dividr el texto en enunciados
nltk.download('stopwords')    # Acceso a "stopwords" en varios idiomas.

[nltk_data] Downloading package punkt to C:\Users\Jorge
[nltk_data]     Pedroza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Jorge
[nltk_data]     Pedroza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Lista de stopwords que se incluyen de manera predeterminada la suite de librerías de NLTK

print(len(stopwords.words('english')))
print(stopwords.words('english'))

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [5]:
#Se extraen los archivos desde repositorio público de GitHub
url1 = 'https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/amazon5.txt'
url2 = 'https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/imdb5.txt'
url3 = 'https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/yelp5.txt'

#Se cargan los archivos en dataframe de Pandas
dfa5 = pd.read_csv(url1, sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi5 = pd.read_csv(url2, sep=' {3,4}', names=['review','label'], header=None, encoding='utf-8')
dfy5 = pd.read_csv(url3, sep='\t', names=['review','label'], header=None, encoding='utf-8')

#Se imprime la forma de los dataframes
print('Total de registros de Amazon_5:',dfa5.shape)
print('Total de registros de IMBD_5:',dfi5.shape)
print('Total de registros de Yelp_5:',dfy5.shape)

C:\Users\Jorge Pedroza\AppData\Local\Temp\ipykernel_28132\3513401339.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfi5 = pd.read_csv(url2, sep=' {3,4}', names=['review','label'], header=None, encoding='utf-8')


Total de registros de Amazon_5: (1000, 2)
Total de registros de IMBD_5: (1000, 2)
Total de registros de Yelp_5: (1000, 2)


In [6]:
dfi5.head()

,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [7]:
#Concatenar los 3,000 registros

df = pd.concat([dfa5, dfi5, dfy5], ignore_index=True)
print('Numero de registros',df.shape)
df.info()

Numero de registros (3000, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


Al realizar la revisión del conjunto de datos se observa que el dataset de **imdb5** tiene 1.000 flotantes. Lo que requiere una limpieza especial para dicho dataset. --- **Sin embargo, se encontró que al cargar con el separador correcto, el siguiente paso no es necesario**.

In [8]:
#dfi5.info()

In [9]:
#no necesario
#print(dfi5_c.isnull().sum())

In [10]:
#no necesario con la limpieza anterior
#print(df['label'].isnull().sum())

In [11]:
#no necesario con la limpieza anterior
#dfii5_l = dfi5.fillna(0)
#dfii5_l.head()

In [12]:
#No necesario con la limpieza anterior
#df = pd.concat([dfa5, dfii5_l, dfy5], ignore_index=True)
#df.info()

In [13]:
X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s

assert X.shape == (3000,)           # verificando que tenemos la dimensiones esperadas.
assert Y.shape == (3000,)

# **Pregunta 2**

Realiza de nuevo un proceso de limpieza. Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización. Como aplicaremos modelos embebidos pre-entrenados, queremos palabras lo más cercanas a las existentes en un idioma, inglés en este caso. Aplica y justifica cualquier otro proceso de limpieza que consideres
adecuado. Recuerda que en esta actividad se usarán vectores embebidos para un problema de clasificación, por lo que deberás tomar de acuerdo a este contexto. Justifica todas las transformaciones que se apliquen.



**Procedimiento para quitar las negaciones del conjunto de stopwords en caso de ser necesario**

In [14]:
# Consideremos la siguiente lista de palabras asociada a negaciones en inglés:

mystopwords = stopwords.words('english')

negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

#Se asigna las stopwords de NLTK a mystopwords
mystopwords_without_neg = stopwords.words('english')

#Se revisa si cada una de las palabras en negwords está presente en mystopwords
for word in negwords:
    if word in mystopwords_without_neg:
      mystopwords_without_neg.remove(word)           # si la palabra está presente, quitarla de la lista

#Se imprime la longitud y elementos de negwords para verificar resultados
print(len(negwords))
print(negwords)

#Se imprime la longitud y elementos de los stop words de NLTK para verificar resultados
print(len(mystopwords))
print(mystopwords)

print(len(mystopwords_without_neg))
print(mystopwords_without_neg)

40
['no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about'

**NOTA**: Es importante tener en cuenta, que en el análisis de sentimientos debemos abarcar el vocabulario suficiente que nos permita entender la opinión del usuario y así mismo identificar aquellas opiniones que puden enriquecer y construir una interacción más amena con el cliente. Para ello, Se considera realizar una depuración de las negaciones que no agregan valor al resultado del diccionario que se requiere y que no definen a profundidad el sentimiento del cliente, estas se pueden ser reemplazadas por un vector continuo más cercano.

In [15]:
def clean_tok(doc):
    ##############################################################################
    # AGREGA AQUÍ TUS LÍNEAS DE CÓDIGO - Pregunta 4:
    
    # Eliminación de signos de puntuación, caracteres especiales.
    
    doc = doc.lower()    #normalización a minúsculas
    
    #Sólo caracteres alfabéticos
    puntuacion = re.sub(r'[^a-z]', ' ', doc)                  #considerar solo caracteres alfabéticos
    puntuacion = re.sub(r'\s{2, }', ' ', puntuacion.strip())  #eliminar todo tipo de espacios que se encuentren
    
    # Tokenizar
    tokenizar = puntuacion.split()                            #tomar el resultado anterior y aplicar método de tokenización a partir del método split
    
    # Eliminación de Stopwords
    
    tokens = [token for token in tokenizar if (token not in mystopwords and len(token) >1)]  #en esta ocasión para elimianr los stopwords se toma la librería de corpus el mismo método.
    
    # FIN PARA AGREGAR TUS LÍNEAS DE CÓDIGO.
    ##############################################################################
    
    return tokens

In [16]:
Xcleantok = [clean_tok(x) for x in X]

In [17]:
df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [18]:
for x in Xcleantok[0:5]:
    print(x)

['way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversations', 'lasting', 'minutes', 'major', 'problems']
['mic', 'great']


# **Método Limpieza por Lematizazión**

In [19]:
#lista vacía para análisis visual
palabras = []

#ciclo para juntar todos los tokens previos en una lista
for tokens1 in Xcleantok:

  palabras.extend(tokens1)

#Set para eliminar repeticiones
dic = set(palabras)

#Imprimir resultados para análisis visual
#print(sorted(dic))


#Se crea un objeto de la clase PorterStemmer
#ps = PorterStemmer() #Se requiere usar lematización para utilizar el método de vectores embebidos

#Se crea un objeto de la clase WordNetLemmatizer
WNL = WordNetLemmatizer()

#Definición de la función de limpieza adicional
def clean_doc(doc):

    #Se define lista vacía para los nuevos tokens
    # tokens = stemmer_tokens(doc)
    clean_tokens = []
    
    #Ciclo para la limpieza de los tokens
    for token in doc:

        no_dups_token = re.sub(r'([a-z])\1{2,}', r'\1\1', token) 
        # Se aplica un filtro con regex para eliminar palabras donde se repita más de 2 veces seguidas la misma letra

        
        if len(no_dups_token) == 2 and no_dups_token.endswith('s'):
            # Se aplica un filtro para las palabras como "as", "is", y "us", que no se les elimine la "s" final por la lematización
            clean_tokens.append(no_dups_token)
            continue
        elif len(no_dups_token) == 3 and no_dups_token.endswith('ed'):
            # Se aplica un filtro para las palabras como "ted", "fed", y "ned", que no se les eliminen las "ed" finales por la lematización
            clean_tokens.append(no_dups_token)
            continue
        elif len(no_dups_token) == 4 and no_dups_token.endswith('ing'):
            # Se aplica un filtro para las palabras como "ping", "bing", y "ring", que no se les eliminen las "ing" finales por la lematización
            clean_tokens.append(no_dups_token)
            continue
            
        #Se aplica stemming lo cual nos llevará las palabras a su base, incluyendo remover las terminaciones 'ing','ed','s'
        #doc[j] = ps.stem(doc[j]) #Se aplica lematización en lugar de stemming
    
        #Sólo lematizar tokens mas de más de dos caracteres
        if len(token) < 2:
            continue
    
        #Se intenta lematizar verbo
        lem_token = WNL.lemmatize(token,'v')
        
        #Si el token permanece sin cambios, intentar lematizar como sustantivo
        if lem_token == no_dups_token:
            lem_token = WNL.lemmatize(token, 'n')
        
        #Si el token permanece sin cambios, intentar lematizar como adjetivo
        if lem_token == no_dups_token:
            lem_token = WNL.lemmatize(token,'a')
        
        #Si el token permanece sin cambios, intentar lematizar como advervio
        if lem_token == no_dups_token:
            lem_token = WNL.lemmatize(token,'r')
        
        #Agregar el resultado a la lista de tokens límpios (clean_tokens)
        clean_tokens.append(lem_token)

    return clean_tokens

In [20]:
# Aplicamos el proceso de limpieza/normalización adicionales:

Xclean = [clean_doc(x) for x in Xcleantok]

Comparamos la información antes y después de la limpieza

In [21]:
Xcleantok[0:10]

[['way', 'plug', 'us', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tied',
  'charger',
  'conversations',
  'lasting',
  'minutes',
  'major',
  'problems'],
 ['mic', 'great'],
 ['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume'],
 ['several',
  'dozen',
  'several',
  'hundred',
  'contacts',
  'imagine',
  'fun',
  'sending',
  'one',
  'one'],
 ['razr', 'owner', 'must'],
 ['needless', 'say', 'wasted', 'money'],
 ['waste', 'money', 'time']]

In [22]:
Xclean[0:10]

[['way', 'plug', 'us', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem'],
 ['mic', 'great'],
 ['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume'],
 ['several',
  'dozen',
  'several',
  'hundred',
  'contact',
  'imagine',
  'fun',
  'send',
  'one',
  'one'],
 ['razr', 'owner', 'must'],
 ['needle', 'say', 'waste', 'money'],
 ['waste', 'money', 'time']]

# **Pregunta 3**

Llamar Xclean a los comentarios procesados y Y a las etiquetas. Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente. Verifica que obtienes 2,100 registros de entrenamiento y 450 para cada uno de validación y prueba.


In [23]:
Xclean[:10]

[['way', 'plug', 'us', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tie', 'charger', 'conversation', 'last', 'minute', 'major', 'problem'],
 ['mic', 'great'],
 ['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume'],
 ['several',
  'dozen',
  'several',
  'hundred',
  'contact',
  'imagine',
  'fun',
  'send',
  'one',
  'one'],
 ['razr', 'owner', 'must'],
 ['needle', 'say', 'waste', 'money'],
 ['waste', 'money', 'time']]

In [24]:
# Xclean = Comentarios procesados
# Y = etiquetas

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)

print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


# **Pregunta 4**

## **Pregunta 4 A**

***Usa el conjunto de entrenamiento para generar tu vocabulario con un tamaño que
consideres adecuado. Puedes filtrar tu vocabulario por la frecuencia mínima de uso de cada
palabra, así como por su longitud mínima en caracteres.***

In [25]:
from collections import Counter

# Definir parámetros de filtrado
frecuencia_minima = 2
longitud_minima = 3

# Contar las palabras en el conjunto de entrenamiento
contador_palabras = Counter()
for comentario in x_train:
    contador_palabras.update(comentario)

# Filtrar el vocabulario
vocabulario = {palabra for palabra, frecuencia in contador_palabras.items()
               if frecuencia >= frecuencia_minima and len(palabra) >= longitud_minima}


## **Pregunta 4 B**

**Indica el tamaño del vocabulario generado.**

In [26]:
# Mostrar el tamaño del vocabulario generado
print(f"Tamaño del vocabulario generado: {len(vocabulario)}")

Tamaño del vocabulario generado: 1401


## **Pregunta 4 C**

**¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?**

Usar solo el conjunto de entrenamiento para generar el vocabulario es importante por las siguientes razones:

Evita la fuga de datos (data leakage): La fuga de datos ocurre cuando la información del conjunto de validación o prueba se usa para crear el modelo, lo que puede dar lugar a un rendimiento inflado que no se reflejará en datos nuevos.
Representación realista: El vocabulario debe reflejar lo que el modelo encontrará en datos no vistos. Usar solo el conjunto de entrenamiento asegura que el modelo no se adapte específicamente a las características de los conjuntos de validación y prueba.
Evaluación justa: Para evaluar de manera justa el rendimiento del modelo, los conjuntos de validación y prueba deben mantenerse independientes de cualquier proceso de modelado que involucre el conjunto de entrenamiento.

## **Pregunta 4 D**


**Con el vocabulario generado, filtra los conjuntos de entrenamiento, validación y prueba para
que todos los comentarios usen solamente las palabras de este vocabulario.**

In [27]:
def filtrar_comentarios(comentarios, vocabulario):
    return [[palabra for palabra in comentario if palabra in vocabulario] for comentario in comentarios]

# Filtrar los conjuntos de datos
x_train_filtrado = filtrar_comentarios(x_train, vocabulario)
x_val_filtrado = filtrar_comentarios(x_val, vocabulario)
x_test_filtrado = filtrar_comentarios(x_test, vocabulario)

# Verificar el tamaño de los conjuntos filtrados
print(f"Tamaño del conjunto de entrenamiento filtrado: {len(x_train_filtrado)}")
print(f"Tamaño del conjunto de validación filtrado: {len(x_val_filtrado)}")
print(f"Tamaño del conjunto de prueba filtrado: {len(x_test_filtrado)}")


Tamaño del conjunto de entrenamiento filtrado: 2100
Tamaño del conjunto de validación filtrado: 450
Tamaño del conjunto de prueba filtrado: 450


# **Pregunta 5**
Utilizarás los vectores embebidos FastText preentrenados por Facebook.

Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford. Puedes consultar sus páginas correspondientes:
- https://fasttext.cc/
- https://code.google.com/archive/p/word2vec/
- https://nlp.stanford.edu/projects/glove/

| Características | FastText | Word2Vec | GloVe |
| ----------- | ----------- | ----------- | ----------- |
| **Institución desarrolladora** | Facebook AI | Google | Stanford University |
| **Modelo** | Basado en la arquitectura de skip-gram y CBOW | Basado en las arquitecturas de skip-gram y CBOW | Basado en la matriz de co-ocurrencia de palabras en el corpus |
| **Manejo de palabras fuera del vocabulario (OOV)** | Buen manejo gracias al uso de n-grams subwords | No lo soporta | No lo soporta |
| **Manejo de palabras raras** | Buen manejo gracias al uso de n-grams subwords | Puede tener dificultades con palabras raras | Buen manejo dependiendo de la frecuencia en la matriz de co-ocurrencia |
| **Velocidad de entrenamiento** | Media | Rápido | Lento en comparación con FastText y Word2Vec debido al uso de la matriz de co-ocurrencia |
| **Dimensiones de los vectores** | Configurable, hasta 300 dimensiones | Configurable, hasta 300 dimensiones | Configurable, hasta 300 dimensiones |
| **Requisitos de memoria** | Moderados, debido a los subwords n-grams | Bajos, sólo necesita palabras y contextos | Altos, debido a la matriz de co-ocurrencia |
| **Interpretación semántica** | Buena, similar a Word2Vec | Buena, basada en contextos cercanos | Buena, basada en relaciones semánticas y sintácticas |


<br>

Es importante mencionar que cada uno de estos modelos puede ser el más adecuado dependiendo de las necesidades específicas del problema que se esté abordando.

Por ejemplo, si necesitas manejar palabras fuera del vocabulario o palabras raras, FastText podría ser la mejor opción; si la memoria es una limitación, Word2Vec podría ser más adecuado; GloVe, por otro lado, podría ser más útil para capturar relaciones semánticas y sintácticas entre palabras.

Por otro lado, GloVe podría desempeñarse mejor que Word2Vec para tareas de analogía de palabras, y también muestra un desempeño superior en tareas de similitud y de reconocimiento de nombre de entidades. Mientras que FastText logra un mejor desempeño en tareas sintácticas. Por último, Word2Vec sobrepasa a FastText en tareas semánticas.


# **Pregunta 6**
Utiliza el modelo **FastText** de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300. Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.
https://fasttext.cc/docs/en/crawl-vectors.html

**NOTA**: Debido a la cantidad de recursos computacionales que demanda cargar los vectores FastText (son 2 millones de vectores), es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo (pickle, npz o el que consideres más adecuado). Una vez realizado lo anterior, puedes borrar la variable de FastText para liberar memoria RAM. De esta manera, ya tienes tu vocabulario de vectores embebidos de acuerdo a los tokens que consideras más adecuados para tu problema y puedes usarlo rápidamente
cuando lo necesites. En dado caso apóyense entre los miembros del equipo de tener dificultades para generar el vocabulario y por mientras puedes usar el archivo del vocabulario que alguno haya generado.

**Se instala el módulo de Cython y FastText**

In [28]:
# !pip install fasttext
# !pip install Cython --install-option="--no-cython-compile"

**Se importa el módulo de fasttext**

In [29]:
import fasttext
import fasttext.util

#Se descargan los el modelo de idioma inglés
fasttext.util.download_model('en', if_exists='ignore')  # English

ModuleNotFoundError: No module named 'fasttext'

In [ ]:
#Se utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
#Copia de midicc para realizar operaciones
midicc_vec = {}

#asignar el vector resultante a cada elemento del diccionario
for word in vocabulario:
    vec_word = ft.get_word_vector(word)
    midicc_vec[word] = vec_word

print('Longitud del vocabulario de vectores embebidos:', len(midicc_vec))
print('Las dimensiones de midicc_vec son: {},{}'.format(len(midicc_vec), len(list(midicc_vec.items())[0][1])))   # veamos algunos elementos del diccionario.
print(list(midicc_vec.items())[0:1])     # veamos algunos elementos del diccionario.

Longitud del vocabulario de vectores embebidos: 1401
Las dimensiones de midicc_vec son: 1401,300
[('oven', array([ 0.07280932, -0.02370244, -0.05169982,  0.16757505,  0.177044  ,
       -0.00364497,  0.08935895,  0.1375987 ,  0.11373347,  0.08866402,
       -0.04330574,  0.09352942,  0.11257228, -0.09670949,  0.19062184,
        0.06032889, -0.16845928,  0.18443851, -0.09148218,  0.00329061,
       -0.13723697,  0.21887699, -0.16951388, -0.01393365,  0.08403366,
       -0.05228687,  0.08536837,  0.02007805,  0.04613523,  0.38683817,
        0.05466906,  0.2839953 , -0.07634079,  0.14579573, -0.01009578,
        0.010638  ,  0.02913181,  0.25452062, -0.19712773, -0.1252984 ,
        0.00705999, -0.00694526, -0.03005858, -0.02225715, -0.30444366,
        0.01189581,  0.01983473,  0.06075285,  0.16756222, -0.07386865,
        0.2288286 ,  0.10335812,  0.11482974, -0.01085357, -0.00372904,
        0.01727128, -0.00283727, -0.05555332, -0.16086215, -0.01426944,
       -0.02262048,  0.106218

In [ ]:
# import fasttext
# import fasttext.util
import pickle

In [ ]:
with open('diccionario_vectores.pkl', 'wb') as archivo:
    pickle.dump(midicc_vec, archivo)

print("Diccionario de vectores embebidos guardado en 'diccionario_vectores.pkl'")

# Liberar memoria
del ft

Diccionario de vectores embebidos guardado en 'diccionario_vectores.pkl'


# **Pregunta 7**
Una manera de utilizar los vectores embebidos con modelos de aprendizaje automático en
documentos de texto, es asignar a cada comentario filtrado el vector embebido de dimensión 300
que resulta de promediar todos sus tokens. Así, en este ejercicio deberás generar los arreglos
correspondientes para los conjuntos de entrenamiento, validación y prueba. Los llamaremos
trainEmb, valEmb y testEmb, respectivamente.

In [ ]:
import pickle
import numpy as np

# Paso 1: Cargar el diccionario de vectores embebidos
with open('diccionario_vectores.pkl', 'rb') as archivo:
    diccionario_vectores = pickle.load(archivo)

# Paso 2: Definir la función para promediar los vectores embebidos de un comentario
def promedio_vectores(comentario, diccionario_vectores, dimension=300):
    vectores = [diccionario_vectores[palabra] for palabra in comentario if palabra in diccionario_vectores]
    if len(vectores) == 0:
        return np.zeros(dimension)
    promedio = np.mean(vectores, axis=0)
    return promedio

# Paso 3: Generar los conjuntos de vectores embebidos promediados
trainEmb = np.array([promedio_vectores(comentario, diccionario_vectores) for comentario in x_train_filtrado])
valEmb = np.array([promedio_vectores(comentario, diccionario_vectores) for comentario in x_val_filtrado])
testEmb = np.array([promedio_vectores(comentario, diccionario_vectores) for comentario in x_test_filtrado])

# Verificar las dimensiones de los conjuntos generados
print(f"Dimensiones de trainEmb: {trainEmb.shape}")
print(f"Dimensiones de valEmb: {valEmb.shape}")
print(f"Dimensiones de testEmb: {testEmb.shape}")


Dimensiones de trainEmb: (2100, 300)
Dimensiones de valEmb: (450, 300)
Dimensiones de testEmb: (450, 300)


# **Pregunta 8**
Utiliza los modelos de regresión lineal y bosque aleatorio (random forest) y encuentra sus
desempeños (accuracy). Compara los resultados con los de la semana anterior.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

## **Random Forest**

In [ ]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Configurar el modelo con validación cruzada
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(trainEmb, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Entrenar el modelo con los mejores hiperparámetros
best_rf = grid_search.best_estimator_

# Realizar predicciones
y_train_pred_best_rf = best_rf.predict(trainEmb)
y_val_pred_best_rf = best_rf.predict(valEmb)

# Evaluar el modelo
accuracy_train_best_rf = accuracy_score(y_train, y_train_pred_best_rf)
accuracy_val_best_rf = accuracy_score(y_val, y_val_pred_best_rf)

print("Bosque Aleatorio con Hiperparámetros Óptimos - Exactitud en el conjunto de entrenamiento:", accuracy_train_best_rf)
print("Bosque Aleatorio con Hiperparámetros Óptimos - Exactitud en el conjunto de validación:", accuracy_val_best_rf)
print("Reporte de clasificación para el conjunto de validación:")
print(classification_report(y_val, y_val_pred_best_rf))


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Mejores hiperparámetros: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Bosque Aleatorio con Hiperparámetros Óptimos - Exactitud en el conjunto de entrenamiento: 0.991904761904762
Bosque Aleatorio con Hiperparámetros Óptimos - Exactitud en el conjunto de validación: 0.7666666666666667
Reporte de clasificación para el conjunto de validación:
              precision    recall  f1-score   support

           0       0.75      0.76      0.76       216
           1       0.78      0.77      0.77       234

    accuracy                           0.77       450
   macro avg       0.77      0.77      0.77       450
weighted avg       0.77      0.77      0.77       450



## **Logistic Regression**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Definir los hiperparámetros a buscar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],  # 'l1' y 'elasticnet' requieren solver diferente
    'solver': ['lbfgs'],  # Puedes usar 'liblinear' para 'l1'
    'max_iter': [100, 200, 300]
}

# Configurar el modelo con validación cruzada
logreg_model = LogisticRegression()
grid_search_logreg = GridSearchCV(estimator=logreg_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_logreg.fit(trainEmb, y_train)

# Obtener los mejores hiperparámetros
best_params_logreg = grid_search_logreg.best_params_
print("Mejores hiperparámetros para regresión logística:", best_params_logreg)

# Entrenar el modelo con los mejores hiperparámetros
best_logreg = grid_search_logreg.best_estimator_

# Realizar predicciones
y_train_pred_best_logreg = best_logreg.predict(trainEmb)
y_val_pred_best_logreg = best_logreg.predict(valEmb)

# Evaluar el modelo
accuracy_train_best_logreg = accuracy_score(y_train, y_train_pred_best_logreg)
accuracy_val_best_logreg = accuracy_score(y_val, y_val_pred_best_logreg)

print("Regresión Logística con Hiperparámetros Óptimos - Exactitud en el conjunto de entrenamiento:", accuracy_train_best_logreg)
print("Regresión Logística con Hiperparámetros Óptimos - Exactitud en el conjunto de validación:", accuracy_val_best_logreg)
print("Reporte de clasificación para el conjunto de validación:")
print(classification_report(y_val, y_val_pred_best_logreg))


Fitting 3 folds for each of 15 candidates, totalling 45 fits
Mejores hiperparámetros para regresión logística: {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Regresión Logística con Hiperparámetros Óptimos - Exactitud en el conjunto de entrenamiento: 0.8461904761904762
Regresión Logística con Hiperparámetros Óptimos - Exactitud en el conjunto de validación: 0.7933333333333333
Reporte de clasificación para el conjunto de validación:
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       216
           1       0.81      0.78      0.80       234

    accuracy                           0.79       450
   macro avg       0.79      0.79      0.79       450
weighted avg       0.79      0.79      0.79       450



*Se obtienen resultados similares a los obtenidos en la semana anterior. El modelo de regresión logística logra exactitudes por encima del 72% y el modelo de Bosques Aleatorios logra resultados similares y conserva su tendencia a sobreentrenarse. Para evitar lo anterios, se probaron varios métodos:*

1. *Implementar validación cruzada.*
2. *Reducir el número de estimadores.*

*El primero no ayudó a reducir la tendencia a sobreentrenar el modelo. El seguno al reducirse a un valor de entr 2 y 10, logró con efectividad reducir el sobreentrenamiento. Sin embargo, la exactitud del conjunto de validación, también resultaba afectada negativamente, por lo tanto, se conservó el modelo sobreentrenado, ya que a pesar de esto, también logra los mejores resultados en el conjunto de validación. Entrenar el modelo con más datos, parece ser la mejor opción para lograr reducir el sobreentrenamiento en el modelo de bosques aleatorios.*

# **Pregunta 9**
Obtener la matriz de confusión e interpretar sus valores.

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_test_pred_best_logreg = best_logreg.predict(testEmb)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_test_pred_best_logreg)

# Calcular el reporte de clasificación
class_report = classification_report(y_test, y_test_pred_best_logreg)

# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

# Mostrar el reporte de clasificación
print("Reporte de clasificación:")
print(class_report)

Matriz de confusión:
[[170  46]
 [ 44 190]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.79      0.79       216
           1       0.81      0.81      0.81       234

    accuracy                           0.80       450
   macro avg       0.80      0.80      0.80       450
weighted avg       0.80      0.80      0.80       450



*Con relación a los resultados de las matrices de confusión por medio de ambos métodos, el modelo de bosques aleatorios obtuvo valor menor de clasificaciones verdaderas positivas, pero mayor en verdaderas negativas, logrando una cantidad neta de clasificaciones correctas mayor que el método de regresión logística. Se logró reducir el número de clasificaciones falsas negativas y se incrementó proporcionalmente el número falsos positivos. Al igual que lo descrito anteriormente, el valor neto clasificaciones incorrectas es menor que por el método de conteo. Lo anterior en resumen, implica que el método de bosques aleatorios es mejor para los algoritmos de clasificación, para diferenciar mejor las clases negativas en general que el método de conteo a cambio de una ligera degradación en la clasificación de clases positivas.*



# **Pregunta 10**
Comenta con tus compañeros de equipo los pasos realizados en esta actividad e incluyan sus conclusiones finales.

>>> *Agregar aquí conclusiones del trabajo*

*Dentro de la limpieza realizada se encontraron algunas validaciones que mejoraron el desempeño del modelo dado que no se procesaron ciertos valores o caracteres que no son relevantes para el estudio y alteran el resultado esperado. Posteriormente, se ejecutaron las particiones de los elementos para el desarrollo de los modelos; para ello, se acotaron los caracteres dentro del diccionario del conjunto de entrenamiento dado que se involucraron únicamente palabras relevantes las cuales tienen una mayor cantidad de frecuencia dentro de los datos.*

*Luego del preprocesamiento anterior, se inició la validación del modelo **FastText** con el fin de identificar la relación que tienen las palabras que contiene el diccionario definido. Luego de inicializarlo, se desarrollaron las funciones requeridas para obtener el promedio de los vectores y de allí otra función que transforme los documentos a vectores embebidos con el fin de asignarles valores a los vectores e identificar la correlación de cada uno entre ellos.*

*Con los procedimientos anteriores, se utilizaron dichos vectores para optimizar los modelos de Random Forest y Linear Regression, obteniendo su desempeño y la validación de estos. Detectando los hiperparámetros requeridos para cada modelo y obteniendo los resultados del desempeño de cada uno. **FastText** permite una mejor predicción dado que asigna una mejor relación entre las palabras que únicamente valores de 1 y 0 para los caracteres obtenidos, distorsionando el valor del desempeño de los modelos.*

*Para este caso, el método de transformar el corpus a sus vectores preentrenados embebidos, después de el proceso de limpieza y lematización, para después calcular el promedio de los vectores para cada comentario, resultó en una desempeño menor que el que se obtuvo para el mismo conjunto de datos y el uso de la matriz dispersa por conteo de frecuencias y de tf-idf. Con lo anterior, se pudo comprender el valor de tener cada palabra representada como un elemento de un espacio vectorial para encontrar la relación entre las palabras cercanas, sin embargo, para este caso, el uso del promedio de dichos vectores no resultó superior en exactitud a los resultados logrados con las matrices dispersas. Posiblemente, los resultados se pueden mejorar si emplea una simplificación del comentario diferente al promedio de los vectores, como lo sería un proceso de reducción tomando en cuenta la relación que tiene una palabra con otra.*



##### **Fuentes bibliográficas y de datos:**
- https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/amazon5.txt
- https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/imdb5.txt
- https://raw.githubusercontent.com/Handrum/NLP_EQ_6/main/yelp5.txt
- https://fasttext.cc/
- https://fasttext.cc/docs/en/crawl-vectors.html
- https://code.google.com/archive/p/word2vec/
- https://nlp.stanford.edu/projects/glove/
- https://medium.com/analytics-vidhya/word2vec-glove-fasttext-and-baseline-word-embeddings-step-by-step-d0489c15d10b

- Vajjala, S., Majumder, B., Gupta, A., y Surana, H. (2020). Practical Natural Language Processing: A Comprehensive Guide to Building Real-World NLP Systems. O'Reilly. https://learning.oreilly.com/library/view/practical-natural-language/9781492054047/


- Khurana, D., Koli, A., Khatter, K., y Singh, S. (2023). Natural language processing: state of the art, current trends and challenges. Multimed Tools Appl 82, 3713–3744. https://link.springer.com/article/10.1007/s11042-022-13428-4Links to an external site.

- Falcón Morales, L. E. (2023). Bolsa de palabras: BOW [PDF]. Maestría en Inteligencia Artificial Aplicada. ITESM. Acceso al material Download Acceso al material